# Web scrapping on breakup forum 

In [3]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import nltk
import language_tool_python
import openai
import csv
import time
from bs4 import BeautifulSoup
import html
import re 
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('brown')



In [4]:
## Function to extract the URL of each  breakup topics  

# set a user-agent that mimics a web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
# Create a session object
session = requests.Session()


def scrape_forum_page(url):
    try:
        response = session.get(url,headers=headers) #  Use the session to make the request
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        thread_links = soup.select('.forum_topic__title a')
        base_url = "https://couple.journaldesfemmes.com"
        thread_urls = [base_url + link['href'] for link in thread_links if 'href' in link.attrs]

        # Find the link to the next page
        next_page_link = soup.find('a', attrs={'rel': 'next'})
        next_page_url = base_url + next_page_link['href'] if next_page_link else None

        return thread_urls, next_page_url
    except requests.RequestException as e:
        print(f"Error: {str(e)}")
        return [], None

# Start with the first page
current_page_url = "https://couple.journaldesfemmes.com/forum/rupture-115"
all_discussion_urls = []

while current_page_url:
    print(f"Scraping: {current_page_url}")
    page_urls, current_page_url = scrape_forum_page(current_page_url)
    all_discussion_urls.extend(page_urls)

print(f"Total Discussions Scraped: {len(all_discussion_urls)}")
print(all_discussion_urls)

Scraping: https://couple.journaldesfemmes.com/forum/rupture-115
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=2
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=3
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=4
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=5
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=6
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=7
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=8
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=9
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=10
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=11
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=12
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=13
Scraping: https://couple.journaldesfemmes.com/forum/rupture-115?page=14
Scraping

In [1]:
#funct to get topics content 

##set a user-agent that mimics a web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

#Utility function to clean text by removing non-breaking spaces and URLs.
def clean_text(text):
    text = html.unescape(text)  # Unescape HTML entities
    text = text.replace(u'\xa0', u' ')  # Replace non-breaking spaces
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    return text



# Function to read already scraped topics
def read_existing_topics(filename):
    existing_topics = set()
    if os.path.exists(filename):
        with open(filename, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader, None)  # Skip header
            for row in reader:
                topic_url = row[0]
                existing_topics.add(topic_url)
    return existing_topics


def scrape_discussion_content(discussion_url):
    try:
        response = requests.get(discussion_url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        original_post = ''
        responses = []

        # Locate the container with the original post content
        original_post_container = soup.find('div', class_='jQdispctn for_topic__question typo_content')
        if original_post_container:
            original_post_content = ' '.join(p.get_text(strip=True) for p in original_post_container.find_all('p') if p.get_text(strip=True))
            original_post = clean_text(original_post_content)
        else:
            print(f"Warning: No original post container found for URL {discussion_url}")

        # Locate the containers with the responses
        response_containers = soup.find_all('div', class_='for_answer__content')
        if response_containers:
            responses = [clean_text(' '.join(p.get_text(strip=True) for p in response.find_all('p') if p.get_text(strip=True))) for response in response_containers]
        else:
            print(f"Warning: No response containers found for URL {discussion_url}")

        return original_post, responses
    except requests.RequestException as e:
        print(f"Error: {str(e)}")
        return "", []


scrape_discussion_content('https://couple.journaldesfemmes.com/forum/affich-70943-comment-oublier-son-ex')



NameError: name 'requests' is not defined

In [5]:
all_discussion_urls_part = all_discussion_urls
print(len(all_discussion_urls_part))
print(all_discussion_urls_part)

768
['https://couple.journaldesfemmes.com/forum/affich-71019-se-remettre-d-un-rupture-quand-on-aime-encore', 'https://couple.journaldesfemmes.com/forum/affich-71063-je-n-arrive-pas-a-l-oublier-elle-me-manque', 'https://couple.journaldesfemmes.com/forum/affich-71060-plaque-a-50-ans', 'https://couple.journaldesfemmes.com/forum/affich-71047-j-ai-tenu-un-mensonge-par-manque-de-vie-sociale', 'https://couple.journaldesfemmes.com/forum/affich-70992-mon-homme-pere-de-mes-enfants-m-a-quittee', 'https://couple.journaldesfemmes.com/forum/affich-70943-comment-oublier-son-ex', 'https://couple.journaldesfemmes.com/forum/affich-70966-se-remettre-d-une-rupture-amoureuse', 'https://couple.journaldesfemmes.com/forum/affich-70952-rupture-complique', 'https://couple.journaldesfemmes.com/forum/affich-70909-on-s-aime-tjrs-follement-mais-on-s-est-quittes', 'https://couple.journaldesfemmes.com/forum/affich-70898-je-souhaiterais-recuperer-mon-ex', 'https://couple.journaldesfemmes.com/forum/affich-70845-sr-pour

## RUN SCRAPPING 

In [5]:
filename = 'scraped_discussions.csv'
existing_topics = read_existing_topics(filename)

with open(filename, mode='a', newline='', encoding='utf-8') as file:  # 'a' mode to append to file
    writer = csv.writer(file)
    if not existing_topics:  # Write header only if file was empty
        writer.writerow(["URL", "Original Post", "Responses"])

    for url in all_discussion_urls_part:
        if url not in existing_topics:
            print('Processing:', url)
            original_post, responses = scrape_discussion_content(url)
            writer.writerow([url, original_post, "|".join(responses)])
            time.sleep(20)  # Rate limiting

print("Scraping complete.")

process :  https://couple.journaldesfemmes.com/forum/affich-71019-se-remettre-d-un-rupture-quand-on-aime-encore
process :  https://couple.journaldesfemmes.com/forum/affich-70992-mon-homme-pere-de-mes-enfants-m-a-quittee
process :  https://couple.journaldesfemmes.com/forum/affich-70943-comment-oublier-son-ex
process :  https://couple.journaldesfemmes.com/forum/affich-70966-se-remettre-d-une-rupture-amoureuse
process :  https://couple.journaldesfemmes.com/forum/affich-70952-rupture-complique
process :  https://couple.journaldesfemmes.com/forum/affich-70909-on-s-aime-tjrs-follement-mais-on-s-est-quittes
process :  https://couple.journaldesfemmes.com/forum/affich-70898-je-souhaiterais-recuperer-mon-ex
process :  https://couple.journaldesfemmes.com/forum/affich-70845-sr-pour-recuperer-son-ex-et-le-convaincre-de-notre-changemen
process :  https://couple.journaldesfemmes.com/forum/affich-70850-est-il-possible-de-revenir-en-arriere
process :  https://couple.journaldesfemmes.com/forum/affich-70

In [56]:
# Clean  conversation 

#tool = language_tool_python.LanguageTool('fr')

def correct_grammar(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# Example usage
original_text = "bsr en ne vivant plus sous le même toit et en reprenant en main sa vie"
corrected_text = correct_grammar(original_text)
print("Corrected Text:", corrected_text)

Corrected Text: bar en ne vivant plus sous le même toit et en reprenant en main sa vie


In [ ]:
# clean conversation using GPT 



def correct_text_with_gpt(text):
    try:
        # Call the OpenAI API to correct the text
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text}
            ],
            temperature=0.5,  # You can adjust temperature to control creativity.
            max_tokens=1024  # Adjust based on how long the input text is.
        )
        
        # Extract the corrected text from the response
        corrected_text = response['choices'][0]['message']['content']
        return corrected_text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
original_text = "The orginal text from your scraped data with errors."
corrected_text = correct_text_with_gpt(original_text)
print("Corrected Text:", corrected_text)
